In [1]:
if __name__=="__main__":
	import sys
	sys.path.append('/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/Scripts')
	
from src.SMT_Analysis_BP.databases.trajectory_analysis_script import *
from scipy.interpolate import griddata
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from scipy.stats import gaussian_kde
import pandas as pd
import os
import numpy as np
from src.SMT_Analysis_BP.helpers.MSD_Utils import *
from src.SMT_Analysis_BP.helpers.plotting_functions import *
from src.SMT_Analysis_BP.helpers.import_functions import *
from src.SMT_Analysis_BP.helpers.Analysis_functions import *
from src.SMT_Analysis_BP.helpers.diff_mw import *
import mpl_toolkits.mplot3d.art3d as art3d
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from src.SMT_Analysis_BP.Random.scalebars import *
from src.SMT_Analysis_BP.Random.Convert_csv_mat import *
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from scipy import stats, ndimage,io
import csv  
from sklearn.cluster import OPTICS
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')  
from sklearn.cluster import DBSCAN
from cgitb import small
import src.SMT_Analysis_BP.helpers.smallestenclosingcircle as smallestenclosingcircle
import math
import src.SMT_Analysis_BP.Random.guassian_fit as gaussian_fit
import src.SMT_Analysis_BP.helpers.fbm_utility as fbm_utility
import src.SMT_Analysis_BP.helpers.simulate_foci as simulate_foci
import src.SMT_Analysis_BP.helpers.simulate_cells as simulate_cells
import src.SMT_Analysis_BP.helpers.SMT_converters as smt
import matplotlib.patches as mpatches
import pickle
import src.SMT_Analysis_BP.Parameter_Store.global_params as param_store

/var/folders/_t/qlzp5l894v16vmr8m_81x5x80000gn/T/ipykernel_1899/2259065141.py:34: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [ ]:
globals = param_store.PIXELSIZES
from warnings import filterwarnings
filterwarnings('ignore',module="DeprecationWarning")
#ignore deprecated warnings
#ignore stacklevel=2 warnings

In [ ]:
def perform_run_analysis_bulk(path_keyword_dict:dict,**kwargs)->list:
    '''
    We will do the run analysis in bulk for a set of data which are repeats of a condition

    Parameters:
    -----------
    path_keyword_dict:dict
        A dictionary of the form {cd:{t_string:_,unique_name:_}}
        CD: is the condition directory
        t_string: is the string which if of internal use and signifies the file naming convention
        unique_name: is the unique name of the condition
    **kwargs:
        Keyword arguments which are passed to the run_analysis function
        TODO update this to be more specific

    Returns:
    --------
    list
        A list of the results of the run_analysis function for each condition.
        Each index is a realization of the trajectory_analysis_script.run_analysis class
    '''
    results = []
    obj= run_analysis("/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/new_days/20190527/ll_ez","laco_laci_ez")
    obj.read_parameters(minimum_percent_per_drop_in = 0.5, 
                        t_len_u = 50, 
                        t_len_l = 2, 
                        minimum_tracks_per_drop = 3)
    
    obj.get_blob_parameters(threshold=1e-4,
                            overlap=0,
                            detection_name='bp',
                            min_sigma=1/np.sqrt(2),
                            max_sigma=5/np.sqrt(2),
                            num_sigma=500,median = False)

    obj.get_fitting_parameters(kwargs={"mask_size":3,
                                        "plot_fit":False,
                                        "fitting_image":"Original",
                                        "radius_func":np.mean,#identity,
                                        "residual_func":residuals_gaus2d,
                                        "sigma_range":0.5,
                                        "centroid_range":0.5,
                                        "height_range":2})
    obj.type_of_blob ="Scale"
    obj.a_file_style = "new"
    obj.run_flow()
    if kwargs.get("save",False):
        obj._make_SMAUG_files(dir_name = "SMAUG_AUTO")
        obj._make_NOBIAS_files(dir_name = "NOBIAS_AUTO")
        obj._store_combined_SMAUG_files(combined_dir_name="/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/Combined_SMAUG/AUTO_STYLE/{}".format())
        

In [ ]:
rpoc_ez_paths_keywords = {"RPOC_EZ":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/Other_RPOC":{"t_string":"rpoc","name":"20180813_Other_RPOC","idiot_factor":1},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/RPOC_new":{"t_string":"RPOC","name":"20181003_RPOC_new","idiot_factor":1},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/new_days/20190527/rpoc_ez":{"t_string":"rpoc_ez","name":"newdays_20190527_rpoc_ez","idiot_factor":1}
    }
}
rpoc_m9_paths_keywords = {"RPOC_M9":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/12/rpoc_m9":{"t_string":"rpoc_m9","name":"12_rpoc_m9","idiot_factor":0.13},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/12/rpoc_m9_2":{"t_string":"rpoc_m9","name":"12_rpoc_m9_2","idiot_factor":0.13},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/rpoc_M9/20190515":{"t_string":"rpoc_M9","name":"20190515_rpoc_m9","idiot_factor":1}
    }
}
rpoc_hex5_paths_keywords = {"RPOC_Hex5":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/15/rp_ez_hex5":{"t_string":"rp_ez_hex5","name":"15_rp_hex5","idiot_factor":1},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/15/rp_ez_hex5_2":{"t_string":"rp_ez_hex5","name":"15_rp_hex5_2","idiot_factor":0.13},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/rp_ez_hex5":{"t_string":"rp_ez_hex5","name":"16_rp_hex5","idiot_factor":1}
    }
}
laci_laco_ez_paths_keywords = {"LL_EZ":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/new_days/20190527/ll_ez":{"t_string":"laco_laci_ez","name":"newdays_20190527_ll_ez","idiot_factor":1},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/17/ll_ez":{"t_string":"ll_ez","name":"17_ll_ez","idiot_factor":1},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/17/ll_ez_2":{"t_string":"ll_ez","name":"17_ll_ez_2","idiot_factor":1}#check
    }
}
laci_laco_m9_paths_keywords = {"LL_M9":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/20200215/ll_m9":{"t_string":"ll_m9","name":"20200215_ll_m9","idiot_factor":1},
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/ll_m9_2":{"t_string":"ll_m9","name":"16_ll_m9_2","idiot_factor":1},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/ll_m9_3":{"t_string":"ll_m9","name":"16_ll_m9_3","idiot_factor":1}#check
    }
}
laci_laco_hex5_paths_keywords = {"LL_Hex5":{
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/ll_hex5":{"t_string":"ll_ez_hex5","name":"16_ll_hex5"},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/ll_hex5_2":{"t_string":"ll_ez_hex5","name":"16_ll_hex5_2"},#check
    "/Users/baljyot/Documents/CODE/GitHub_t2/Baljyot_EXP_RPOC/DATA/16/ll_hex5_3":{"t_string":"ll_ez_hex5","name":"16_ll_hex5_3"}#check
}
}
    